In [42]:
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
from datetime import datetime
import chardet

In [43]:
import chardet

# Read only a portion of the file to speed up detection
with open('PPR-ALL.csv', 'rb') as f:
    raw_data = f.read(10000)  # Read the first 10,000 bytes
    result = chardet.detect(raw_data)

# Use the detected encoding
df = pd.read_csv('PPR-ALL.csv', encoding=result['encoding'], low_memory=False)


In [44]:
result['encoding']

'Windows-1252'

In [45]:
df = pd.read_csv('PPR-ALL.csv',encoding="Windows-1252")
df.head()

/var/folders/55/j0v2zlbn1575rkmztsycxlfw0000gn/T/ipykernel_44912/1864568991.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('PPR-ALL.csv',encoding="Windows-1252")


,Date of Sale (dd/mm/yyyy),Address,County,Eircode,Price (€),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description
0,01/01/2010,"5 Braemor Drive, Churchtown, Co.Dublin",Dublin,NaN,"€343,000.00",No,No,Second-Hand Dwelling house /Apartment,NaN
1,03/01/2010,"134 Ashewood Walk, Summerhill Lane, Portlaoise",Laois,NaN,"€185,000.00",No,Yes,New Dwelling house /Apartment,greater than or equal to 38 sq metres and less...
2,04/01/2010,"1 Meadow Avenue, Dundrum, Dublin 14",Dublin,NaN,"€438,500.00",No,No,Second-Hand Dwelling house /Apartment,NaN
3,04/01/2010,"1 The Haven, Mornington",Meath,NaN,"€400,000.00",No,No,Second-Hand Dwelling house /Apartment,NaN
4,04/01/2010,"11 Melville Heights, Kilkenny",Kilkenny,NaN,"€160,000.00",No,No,Second-Hand Dwelling house /Apartment,NaN


In [46]:
df.columns

Index(['Date of Sale (dd/mm/yyyy)', 'Address', 'County', 'Eircode',
       'Price (€)', 'Not Full Market Price', 'VAT Exclusive',
       'Description of Property', 'Property Size Description'],
      dtype='object')

In [47]:
df['Property Size Description'].unique()

array([nan,
       'greater than or equal to 38 sq metres and less than 125 sq metres',
       'greater than 125 sq metres', 'less than 38 sq metres',
       'greater than or equal to 125 sq metres',
       'níos mó ná nó cothrom le 38 méadar cearnach agus níos lú ná 125 méadar cearnach',
       'n?os l? n? 38 m?adar cearnach'], dtype=object)

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 681503 entries, 0 to 681502
Data columns (total 9 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   Date of Sale (dd/mm/yyyy)  681503 non-null  object
 1   Address                    681503 non-null  object
 2   County                     681503 non-null  object
 3   Eircode                    156334 non-null  object
 4   Price (€)                  681503 non-null  object
 5   Not Full Market Price      681503 non-null  object
 6   VAT Exclusive              681503 non-null  object
 7   Description of Property    681503 non-null  object
 8   Property Size Description  52829 non-null   object
dtypes: object(9)
memory usage: 46.8+ MB


In [49]:
df = df.drop('Property Size Description', axis=1)

In [50]:
def clean_price(string):
    string = string.replace(",","")
    return float(string[1:])
Price = df['Price (€)'].apply(clean_price)

In [51]:
df = df.drop('Price (€)',axis=1)
df['Price'] = Price

In [52]:
df.rename(columns = {"Date of Sale (dd/mm/yyyy)":"Date"},inplace=True)

In [53]:
replacement_dict = {
    'Second-Hand Dwelling house /Apartment':'Second-Hand',
    'New Dwelling house /Apartment':'New',
    'Teach/Árasán Cónaithe Atháimhe':'Second-Hand',
    'Teach/Árasán Cónaithe Nua':'New',
    'Teach/?ras?n C?naithe Nua':'New'
}
df['Description of Property'] = df['Description of Property'].replace(replacement_dict)
df = df[['Date','Address','County','Eircode','Price','Not Full Market Price','VAT Exclusive','Description of Property']]
df.head()

,Date,Address,County,Eircode,Price,Not Full Market Price,VAT Exclusive,Description of Property
0,01/01/2010,"5 Braemor Drive, Churchtown, Co.Dublin",Dublin,NaN,343000.0,No,No,Second-Hand
1,03/01/2010,"134 Ashewood Walk, Summerhill Lane, Portlaoise",Laois,NaN,185000.0,No,Yes,New
2,04/01/2010,"1 Meadow Avenue, Dundrum, Dublin 14",Dublin,NaN,438500.0,No,No,Second-Hand
3,04/01/2010,"1 The Haven, Mornington",Meath,NaN,400000.0,No,No,Second-Hand
4,04/01/2010,"11 Melville Heights, Kilkenny",Kilkenny,NaN,160000.0,No,No,Second-Hand


In [54]:
Town = []
for i in range(len(df)):
    if df['Address'].loc[i].split(',')[-1][1:].split(" ")[-1].split('.')[-1].lower() == df['County'].loc[i].lower():
        if any(char.isdigit() for char in df['Address'].loc[i].split(',')[-2].strip()):
            string = re.sub(r"\d+","", df['Address'].loc[i].split(',')[-2].strip())
            Town.append(string.lower())
        else:
            Town.append(df['Address'].loc[i].split(',')[-2].strip().lower())
    else:
        Town.append(df['Address'].loc[i].split(',')[-1].strip().lower())

In [55]:
df['Town'] = Town

In [56]:
df.head()

,Date,Address,County,Eircode,Price,Not Full Market Price,VAT Exclusive,Description of Property,Town
0,01/01/2010,"5 Braemor Drive, Churchtown, Co.Dublin",Dublin,NaN,343000.0,No,No,Second-Hand,churchtown
1,03/01/2010,"134 Ashewood Walk, Summerhill Lane, Portlaoise",Laois,NaN,185000.0,No,Yes,New,portlaoise
2,04/01/2010,"1 Meadow Avenue, Dundrum, Dublin 14",Dublin,NaN,438500.0,No,No,Second-Hand,dublin 14
3,04/01/2010,"1 The Haven, Mornington",Meath,NaN,400000.0,No,No,Second-Hand,mornington
4,04/01/2010,"11 Melville Heights, Kilkenny",Kilkenny,NaN,160000.0,No,No,Second-Hand,melville heights


In [61]:
d14 = df[df['Town'] == 'dublin 14']
d18 = df[df['Town'] == 'dublin 18']
d22 = df[df['Town'] == 'dublin 22']

In [62]:
d14.sort_values(by='Price', ascending=False)

,Date,Address,County,Eircode,Price,Not Full Market Price,VAT Exclusive,Description of Property,Town
399114,13/11/2019,"Fernbank, Churchtown Road, Dublin 14",Dublin,NaN,79306927.0,No,Yes,New,dublin 14
160310,25/06/2015,"Apartments 1-80 Dundrum View, Dundrum, Dublin 14",Dublin,NaN,28050000.0,No,No,Second-Hand,dublin 14
280304,27/10/2017,"11 MOYOLA COURT, CHURCHTOWN RD LOWER, DUBLIN 14",Dublin,NaN,4100000.0,Yes,No,Second-Hand,dublin 14
230227,23/11/2016,"76 BUTTERFIELD AVE, RATHFARNHAM, DUBLIN 14",Dublin,NaN,3750000.0,No,No,Second-Hand,dublin 14
230237,23/11/2016,"86 BUTTERFIELD AVENUE, RATHFARNHAM, DUBLIN 14",Dublin,NaN,3750000.0,No,No,Second-Hand,dublin 14
...,...,...,...,...,...,...,...,...,...
173578,01/10/2015,"13 BERWICK HALL, CHURCHTOWN, DUBLIN 14",Dublin,NaN,10158.0,No,No,Second-Hand,dublin 14
247415,23/03/2017,"42A DRUMMARTIN PARK, GOATSTOWN, DUBLIN 14",Dublin,NaN,10000.0,No,No,Second-Hand,dublin 14
118547,29/08/2014,"140 LORETO AVE, RATHFARNHAM, DUBLIN 14",Dublin,NaN,10000.0,Yes,No,Second-Hand,dublin 14
46310,21/05/2012,"60 Willbrook, Rathfarnham, Dublin 14",Dublin,NaN,6000.0,No,No,Second-Hand,dublin 14


In [63]:
d18.sort_values(by='Price', ascending=False)

,Date,Address,County,Eircode,Price,Not Full Market Price,VAT Exclusive,Description of Property,Town
416028,27/02/2020,"189 APARTMENTS AT ELMFIELD, BALLYOGAN, DUBLIN 18",Dublin,NaN,99762972.37,No,No,Second-Hand,dublin 18
613121,25/07/2023,"Glencairn Gate, DUBLIN 18",Dublin,NaN,15727274.09,Yes,Yes,New,dublin 18
653205,12/03/2024,"1-24 Oak House Kiltiernan Wood, Enniskerry Ro...",Dublin,NaN,8948575.61,No,Yes,New,dublin 18
566362,24/10/2022,"1 CLAREMONT PINES, FOXROCK, DUBLIN 18",Dublin,D18K6Y4,3650000.00,No,No,Second-Hand,dublin 18
553026,10/08/2022,"10 BRENNANSTOWN VALE, CARRICKMINES, DUBLIN 18",Dublin,D18K8N6,3500000.00,No,No,Second-Hand,dublin 18
...,...,...,...,...,...,...,...,...,...
109294,17/06/2014,"66 BLOCK O BRACKEN HILL, SANDYFORD, DUBLIN 18",Dublin,NaN,32698.54,No,No,Second-Hand,dublin 18
210336,08/07/2016,"101A WEDGEWOOD, SANDYFORD, DUBLIN 18",Dublin,NaN,25000.00,No,No,Second-Hand,dublin 18
485119,02/07/2021,"67 SHANGANAGH GROVE, SHANKILL, DUBLIN 18",Dublin,D18XK52,10000.00,No,No,Second-Hand,dublin 18
426570,05/06/2020,"2 WOODSIDE HALL, TICKNOCK HILL, DUBLIN 18",Dublin,NaN,9900.00,Yes,No,Second-Hand,dublin 18


In [64]:
d22.sort_values(by='Price', ascending=False)

,Date,Address,County,Eircode,Price,Not Full Market Price,VAT Exclusive,Description of Property,Town
239674,23/01/2017,"1 Milford Manor Green, Clondalkin, Dublin 22",Dublin,NaN,5525000.00,No,Yes,New,dublin 22
460923,25/01/2021,"1-6 Station Grove, Clondalkin, Dublin 22",Dublin,NaN,1315000.00,No,Yes,New,dublin 22
410210,14/01/2020,"9TH LOCK ROAD, CLONDALKIN, DUBLIN 22",Dublin,NaN,1200000.00,No,No,Second-Hand,dublin 22
382382,07/08/2019,"9TH LOCK ROAD, CLONDALKIN, DUBLIN 22",Dublin,NaN,1150000.00,No,No,Second-Hand,dublin 22
540920,01/06/2022,"BALLYBANE, CLONDALKIN, DUBLIN 22",Dublin,D22F625,950000.00,No,No,Second-Hand,dublin 22
...,...,...,...,...,...,...,...,...,...
123322,06/10/2014,"15 YELLOW MEADOWS VALE, CLONDALKIN, DUBLIN 22",Dublin,NaN,13655.35,Yes,No,Second-Hand,dublin 22
503303,20/10/2021,"10 MONASTERY WALK, CLONDALKIN, DUBLIN 22",Dublin,D22E685,12000.00,Yes,No,Second-Hand,dublin 22
191818,11/02/2016,"76 BOOT RD, CLONDALKIN, DUBLIN 22",Dublin,NaN,10476.00,No,No,Second-Hand,dublin 22
466100,26/02/2021,"36 HARELAWN GREEN, CLONDALKIN, DUBLIN 22",Dublin,NaN,10000.00,Yes,No,Second-Hand,dublin 22


In [81]:
carlow = df[df['County']=='Carlow']
carlow.sort_values(by='Price',ascending=False)

,Date,Address,County,Eircode,Price,Not Full Market Price,VAT Exclusive,Description of Property,Town
454419,14/12/2020,"1-20 Maple Park & 11-18 Maple Road, Rivercourt...",Carlow,NaN,5101321.59,No,Yes,New,rivercourt
513365,10/12/2021,"1-22 The Millrace, Burrin Road, Carlow",Carlow,NaN,4473184.14,No,Yes,New,burrin road
530881,30/03/2022,"1-10 Beech Park and 1-9 Beech Close, Rivercour...",Carlow,NaN,3917600.00,No,Yes,New,rivercourt
515937,17/12/2021,"66 Carrigbrook, Tullow Road, Carlow",Carlow,NaN,2940569.94,Yes,Yes,New,tullow road
475854,04/05/2021,"43 Carrigbrook, Tullow Road, Carlow",Carlow,NaN,2389213.07,Yes,Yes,New,tullow road
...,...,...,...,...,...,...,...,...,...
663489,20/05/2024,"KELLISTOWN, CARLOW",Carlow,NaN,9247.00,Yes,No,Second-Hand,kellistown
71176,26/04/2013,"11 dublin road, Carlow, Carlow",Carlow,NaN,9000.00,No,No,Second-Hand,carlow
601391,19/05/2023,"2 STRAWHALL, ATHY ROAD, CARLOW",Carlow,R93C1K6,8557.45,Yes,No,Second-Hand,athy road
56703,16/10/2012,"Raheen, Tobinstown, Tullow",Carlow,NaN,8000.00,No,No,Second-Hand,tullow


In [83]:
carlow = carlow[carlow['Town'] == 'carlow']
carlow.sort_values(by='Price',ascending=False)

,Date,Address,County,Eircode,Price,Not Full Market Price,VAT Exclusive,Description of Property,Town
565494,18/10/2022,"RUTLAND, CARLOW, CO CARLOW",Carlow,R93K6E4,735000.0,No,No,Second-Hand,carlow
676165,31/07/2024,"KYLE PARK, CARLOW, CARLOW",Carlow,NaN,395000.0,No,No,Second-Hand,carlow
589112,01/03/2023,"KILLESHIN RD, CARLOW, CARLOW",Carlow,R93A4C0,395000.0,No,No,Second-Hand,carlow
371492,04/06/2019,"1 MONTGOMERY ST, CARLOW, CO CARLOW",Carlow,NaN,380000.0,No,No,Second-Hand,carlow
651605,29/02/2024,"51 MONACURRAGH, CARLOW, CARLOW",Carlow,R93X889,360000.0,No,No,Second-Hand,carlow
...,...,...,...,...,...,...,...,...,...
161636,03/07/2015,"84 ST MARYS PARK, CARLOW, CARLOW",Carlow,NaN,25000.0,No,No,Second-Hand,carlow
161638,03/07/2015,"85 ST MARYS PARK, CARLOW, CARLOW",Carlow,NaN,25000.0,No,No,Second-Hand,carlow
157121,04/06/2015,"16 MOUNTGOMERY STREET, CARLOW, CO. CARLOW",Carlow,NaN,20000.0,No,No,Second-Hand,carlow
452960,08/12/2020,"5 LOWER STAPLESTOWN RD, CARLOW, CARLOW",Carlow,NaN,16667.0,No,No,Second-Hand,carlow
